In [ ]:
import json
import os

from dotenv import load_dotenv

from api_client import get_and_save, post_and_save
from image_util import base64_image

In [ ]:
load_dotenv()

SHOP_SHOPIFY_ID = os.getenv("SHOP_SHOPIFY_ID")
IMAGE_DBM_01_ID = os.getenv("IMAGE_DBM_01_ID")

In [ ]:
# My Shops
get_and_save("/v1/shops.json", "MyShops")

In [ ]:
# Blueprints
get_and_save("/v1/catalog/blueprints.json", "Blueprints")

In [ ]:
# Blueprint Provider
get_and_save("/v1/catalog/blueprints/706/print_providers.json", "Blueprint_706_Providers")

In [ ]:
# Upload Image
image_file_name = "DBM_01"
body = {
    "file_name": f"{image_file_name}.jpg",
    "contents": base64_image(f"{image_file_name}.jpg")
}
post_and_save("/v1/uploads/images.json", body, f"Image_{image_file_name}")

In [ ]:
# Shipping Cost
get_and_save("/v1/catalog/blueprints/706/print_providers/42/shipping.json", "PrintProvider_42_ShippingCosts")

In [ ]:
# Variants
get_and_save("/v1/catalog/blueprints/706/print_providers/42/variants.json", "PrintProvider_42_Variants")

In [ ]:
# Loading Variants
with open("Saves/PrintProvider_42_ShippingCosts.json", "r") as file:
    cost_content = json.loads(file.read())

variants = []
for profile in cost_content["profiles"]:
    price = profile["first_item"]["cost"]
    for variant_id in profile["variant_ids"]:
        variants.append({
            "id": variant_id,
            "price": price,
            "is_enabled": True
        })

In [ ]:
# Loading Print Areas
with open("Saves/PrintProvider_42_Variants.json", "r") as file:
    variant_content = json.loads(file.read())

print_areas = [{
    "variant_ids": [variant["id"] for variant in variant_content["variants"]],
    "placeholders": [{
        "position": "front",
        "images": [{
            "id": f"{IMAGE_DBM_01_ID}",
            "x": 0.5,
            "y": 0.5,
            "scale": 0.7,
            "angle": 0
        }]
    }]
}]

In [ ]:
# Create Product
body = {
    "title": "T-Shirt DBM!",
    "description": "Funny DBM! T-Shirt",
    "blueprint_id": 706,
    "print_provider_id": 42,
    "variants": variants,
    "print_areas": print_areas
}
post_and_save(f"/v1/shops/{SHOP_SHOPIFY_ID}/products.json", body, "Product_DBM_01")